# 共起語分析スクリプト(Matcher)

このスクリプトはspaCyライブラリを使用して事前に解析されたテキストデータに対して共起語分析を行う。文単位での共起語分析、共起範囲指定による分析、例文抽出機能などが含まれる。

## 使用方法

### 解析データの読み込み

1. 解析済みのテキストデータ（.spacyファイル）とそれに対応するテキストファイル（.txt）を用意する。
2. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `main_directory`: 親ディレクトリへの相対パス  
 - `subdirectories`: 読み込む子ディレクトリ名のリスト 
 - `nlp_model`: 解析に利用したモデル名
 - `selected_file_names`: 特定のファイルのみを読み込む場合はここにtxtファイル名を追加（空リストの場合は全ファイルを読み込む）

### 共起語分析（文単位）の実行

1. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `pattern`: キーワードの検索パターンを指定
 - `output_pos`: フィルターする品詞のリスト
 - `max_output`: 出力する結果の最大数
 - `output_file`: 共起語分析の結果を保存するファイル名
2. スクリプトを実行すると、指定したファイル名でCSVファイルが出力される。

### 共起語分析（共起範囲指定）の実行

1. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `pattern`: キーワードの検索パターンを指定
 - `range_left`, `range_right`: 共起語の左右の範囲を指定
 - `output_pos`: フィルターする品詞のリスト
 - `max_output`: 出力する結果の最大数
 - `lowercase`: キーワードの集計を小文字で行うかどうか
 - `output_file`: 共起語分析の結果を保存するファイル名
2. スクリプトを実行すると、指定したファイル名でCSVファイルが出力される。

### キーワードや周辺トークンの集計

1. 共起語分析（共起範囲指定）の実行後、キーワードとその周辺のトークンの相対インデックスごとの出現回数が集計される。
2. 集計結果は`output_filename`で指定したCSVファイルに出力される。

### 例文抽出機能の実行

1. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `lemma_to_extract`: 抽出する単語を指定
 - `pos_to_extract`: 抽出する単語の品詞を指定
 - `output_text_file`: 抽出した文を保存するファイル名 
2. スクリプトを実行すると、指定した単語と品詞に対応する例文がテキストファイルに保存される。

## 分析内容

### 1. 共起語分析（文単位） 

指定したキーワードパターンに基づいて、文単位での共起語を分析する。結果はCSVファイルとして出力され、全体の共起語と指定した品詞ごとの共起語が含まれる。

### 2. 共起語分析（共起範囲指定）

キーワードを中心に指定した範囲内の共起語を分析する。結果はCSVファイルとして出力され、全体の共起語と指定した品詞ごとの共起語が含まれる。また、キーワードとその周辺のトークンの相対インデックスごとの出現回数が集計され、CSVファイルに出力される。

### 3. キーワードの周辺トークンの集計
共起語分析（共起範囲指定）の結果から、キーワードとその周辺のトークンの相対インデックスごとの出現回数を集計する。集計結果はCSVファイルとして出力され、各相対インデックスにおけるトークンとその出現頻度が含まれる。

### 4. 例文抽出機能

共起語分析の結果から、指定した単語と品詞に対応する例文を抽出し、テキストファイルとして保存する。

# **ライブラリのインポート(最初に実行)**

In [1]:
import spacy
import os
import pandas as pd
from collections import Counter
from spacy.tokens import DocBin
from spacy.matcher import Matcher
from collections import defaultdict

# **解析データの読み込み**
## ユーザーが設定するパラメーター

In [2]:
# 基本設定
main_directory = "processed_data"  # 親ディレクトリへの相対パス
subdirectories = ["wiki_english"]  # 読み込む子ディレクトリ名のリスト
nlp_model = "en_core_web_sm"  # 解析に利用したモデル名

# 特定のファイルのみを読み込む場合は以下のリストにtxtファイル名を追加(空リストの場合は全ファイルを読み込む)
selected_file_names = []

## 解析データ読み込みの実行

In [ ]:
# spaCyの言語モデルをロード
nlp = spacy.load(nlp_model)

# 解析結果を格納する辞書の初期化
docs_dict = {}
# フィルタリングされたDocオブジェクトを格納する辞書の初期化
filtered_docs = {}

# 指定されたサブディレクトリ内のファイルを読み込む
for subdir in subdirectories:
    directory = os.path.join(main_directory, subdir)
    
    # サブディレクトリ内のファイルを読み込む
    for filename in os.listdir(directory): 
        if filename.endswith(".spacy"):  # .spacyファイルならば
            spacy_path = os.path.join(directory, filename)
            txt_filename = filename.replace(".spacy", ".txt")  # .spaCyファイルに対応するテキストファイル名を作成
            txt_path = os.path.join(directory, txt_filename)  # .spacyファイルに対応するテキストファイルのパス
            
            # .spacyファイルからDocオブジェクトを読み込む
            doc_bin = DocBin().from_disk(spacy_path)  # DocBinオブジェクトを読み込む
            docs = list(doc_bin.get_docs(nlp.vocab))  # Docオブジェクトをリストに格納
            
            # 対応する.txtファイルからファイル名を読み込む
            if os.path.exists(txt_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    file_names = [line.strip() for line in f.readlines()]  # ファイル名をリストに格納
                
                # Docオブジェクトとファイル名の対応を辞書に格納
                for doc, fname in zip(docs, file_names):
                    docs_dict[fname] = doc
            else:
                print(f"警告: {txt_filename} に対応するテキストファイルが存在しません。")

# ユーザーがファイル名を指定した場合、それに対応するDocオブジェクトをフィルタリング
if selected_file_names:
    missing_files = []  # 存在しないファイル名を格納するリスト
    for fname in selected_file_names:
        if fname in docs_dict:  # 指定されたファイル名が読み込んだデータに存在する場合
            filtered_docs[fname] = docs_dict[fname]
        else:
            missing_files.append(fname)
    
    if missing_files:
        print(f"警告: 次の指定されたファイルは存在しません: {', '.join(missing_files)}")
    if filtered_docs:
        print("指定された一部のデータが読み込まれました。")
    else:
        print("指定されたファイルに対応するデータが見つかりませんでした。")
else:
    filtered_docs = docs_dict
    print("全てのデータが読み込まれました。")
print(f"読み込まれたデータ数: {len(filtered_docs)}")
print(f"読み込まれたファイル名: {list(filtered_docs.keys())}")

# **共起語分析（文単位）**
## ユーザーが設定するパラメーター

In [ ]:
# キーワードの検索パターンを指定
pattern = [{"LOWER": "hello"}, {"LOWER": "world"}]

# 出力設定
output_pos = ["NOUN", "VERB", "ADJ", "PROPN", "ADV"]   # フィルターする品詞をリストで設定
max_output = 150  # 出力する結果の最大数を設定
output_file = "sentence_level_collocations.csv"  # 出力ファイル名

## 共起語分析の実行（文単位）

In [ ]:
def perform_collocation_search(doc, pattern, collocation_dict, file_name):
    """
    指定されたパターンに基づいて、文単位での共起語を検索し、共起語の情報を辞書に記録する。

    Args:
        doc (Doc): 解析対象のDocオブジェクト
        pattern (list): 検索パターンのリスト
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        file_name (str): 解析対象のファイル名
    """
    matcher = Matcher(nlp.vocab)
    matcher.add("PATTERN", [pattern])
    matches = matcher(doc)

    for sent in doc.sents:  # 文ごとに処理
        spans = [(start, end) for match_id, start, end in matches if sent.start <= start < sent.end]

        if spans:  # キーワードが含まれる文のみ処理
            # キーワードに一致するスパンを除外してトークンをフィルタリング
            filtered_tokens = [token for token in sent if not any(span[0] <= token.i < span[1] for span in spans)]
            record_collocations(filtered_tokens, collocation_dict, file_name)

def record_collocations(tokens, collocation_dict, file_name):
    """
    共起語の情報を辞書に記録する。

    Args:
        tokens (list): 共起語のトークンのリスト
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        file_name (str): 解析対象のファイル名
    """
    for token in tokens:
        key = (token.lemma_, token.pos_)  # 共起語のキーとして、基本形と品詞を組み合わせる
        collocation_dict[key]['lemma'] = token.lemma_  
        collocation_dict[key]['pos'] = token.pos_  
        collocation_dict[key]['freq'] += 1  
        collocation_dict[key]['indexes'].append((file_name, token.i)) 


def save_collocations_to_csv(collocation_dict, output_file, output_pos=None, max_output=None):
    """
    共起語の情報をCSVファイルに保存する。

    Args:
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        output_file (str): 出力ファイル名
        output_pos (list, optional): 出力する品詞のリスト。デフォルトはNone。
        max_output (int, optional): 出力する結果の最大数。デフォルトはNone。
    """
    # 辞書からデータフレームを作成
    df = pd.DataFrame.from_dict(collocation_dict, orient='index')
    
    # 全品詞に対するソート
    df_sorted_all = df.sort_values(by='freq', ascending=False)
    if max_output is not None:
        df_sorted_all = df_sorted_all.head(max_output)
    
    # 指定された品詞でフィルタリング
    dfs_filtered = []
    if output_pos:
        for pos in output_pos:
            df_filtered = df[df['pos'] == pos].sort_values(by='freq', ascending=False)
            if max_output is not None:
                df_filtered = df_filtered.head(max_output)
            df_filtered = df_filtered[['lemma', 'freq']].reset_index(drop=True)
            df_filtered.columns = [f'lemma_{pos}', f'freq_{pos}']
            dfs_filtered.append(df_filtered)
    
    # 指定された品詞に基づくデータフレームを結合
    df_filtered_combined = pd.concat(dfs_filtered, axis=1)
    # 全品詞の共起語データを追加
    df_final = pd.concat([df_filtered_combined, df_sorted_all[['lemma', 'pos', 'freq']].reset_index(drop=True)], axis=1)
    
    # CSVとして保存
    df_final.to_csv(output_file, index=False)

# 共起語を格納するための辞書の初期化
collocation_dict = defaultdict(lambda: {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []})

# 全てのDocオブジェクトに対して共起語分析を実行
for file_name, doc in filtered_docs.items():
    perform_collocation_search(doc, pattern, collocation_dict, file_name) # 共起語分析を実行

# 共起語の総数を計算し、結果を表示
total_matches = len(collocation_dict) # 共起語の総数を計算
if total_matches > 0:
    print(f"分析結果: {total_matches} 件の共起語が見つかりました。")
    save_collocations_to_csv(collocation_dict, output_file, output_pos, max_output)  # 結果をCSVファイルに保存
    print(f"結果が {output_file} に保存されました。")
else:
    print("結果が見つかりませんでした。")

# 例文抽出機能
## ユーザーが設定するパラメーター

In [ ]:
lemma_to_extract = 'earth'  # 抽出する単語を指定
pos_to_extract = 'NOUN'  # 抽出する単語の品詞を指定
output_text_file = 'extract_file.txt'  # 抽出した文を保存するファイル名

## 例文抽出の実行

In [ ]:
def extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma, pos, output_file):
    """
    指定された共起語と品詞に対応する文を抽出し、テキストファイルに保存する。

    Args:
        filtered_docs (dict): フィルタリングされたDocオブジェクトの辞書
        collocation_dict (defaultdict): 共起語情報を格納する辞書(例 : {(lemma, pos): {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []}, ...}
        lemma (str): 抽出する単語のレンマ
        pos (str): 抽出する単語の品詞
        output_file (str): 抽出した文を保存するファイル名
    """
    # 共起語と品詞に対応するインデックスを取得(例 : indexes = [(file_name, token_index), ...]
    indexes = collocation_dict.get((lemma, pos), {}).get('indexes', [])
    if not indexes:
        print(f"警告: 指定された共起語 ({lemma}, {pos}) に対応する例文が見つかりませんでした。")
        return
    
    # 処理された文を追跡するためのセット
    processed_sentences = set()

    # ドキュメントとテキスト情報のペアを用いて実際の文を抽出
    sentences = []
    for file_name, doc in filtered_docs.items():
        for token in doc:
            # トークンのインデックスと文書のテキスト情報を組み合わせたキーを作成
            token_key = (file_name, token.i)
            if token_key in indexes:
                sentence_text = token.sent.text
                # 重複を避けるために、すでに処理された文をチェック
                if sentence_text not in processed_sentences:
                    sentences.append(sentence_text)
                    processed_sentences.add(sentence_text)
    
    # テキストファイルに1行1文の形で出力
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            f.write(sentence + '\n')
    print(f"抽出された文({len(sentences)}件)が {output_file} に保存されました。")

# 文を抽出してテキストファイルに保存
extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma_to_extract, pos_to_extract, output_text_file)

# **共起語分析（共起範囲指定）**
## ユーザーが設定するパラメーター

In [4]:
# キーワードの検索パターンを指定
pattern = [{"LOWER": "hello"}, {"LOWER": "world"}]

range_left = 15    # 共起語の左側の範囲を指定
range_right = 15   # 共起語の右側の範囲を指定

# 出力設定
output_pos = ["NOUN", "VERB", "ADJ", "PROPN", "ADV"]   # フィルターする品詞をリストで設定
max_output = 150  # 出力する結果の最大数を設定
lowercase = True  # 「キーワードや周辺トークンの集計」を小文字で統一するか(True/False)
output_file = "collocations_within_range.csv"  # 出力ファイル名

## 共起語分析の実行

In [ ]:
def search_with_window(doc, pattern, collocation_dict, file_name, range_left, range_right, lowercase=True):
    """
    指定された範囲内で共起語を検索し、共起語の情報を辞書に記録する。

    Args:
        doc (Doc): 解析対象のDocオブジェクト
        pattern (list): 検索パターンのリスト
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        file_name (str): 解析対象のファイル名
        range_left (int): 共起語の左側の範囲
        range_right (int): 共起語の右側の範囲
        lowercase (bool): キーワードの集計を小文字で行うかどうか。デフォルトはTrue。

    Returns:
        defaultdict: トークンの相対インデックスごとの出現回数をカウントする辞書
    """
    matcher = Matcher(nlp.vocab)
    matcher.add("PATTERN", [pattern])
    matches = matcher(doc)
    
    # token_index_countsを初期化
    token_index_counts = defaultdict(lambda: defaultdict(int))
    
    # マッチした各部分に対する処理
    for match_id, start, end in matches:
        span = doc[start:end]  # マッチした部分のスパン
        sentence = span.sent  # スパンが含まれる文
        
        # マッチしたフレーズの開始トークンと終了トークンを基準にウィンドウサイズを適用
        window_start = max(span.start - range_left, sentence.start) # 文の先頭を超えないようにする
        window_end = min(span.end + range_right, sentence.end) # 文の末尾を超えないようにする
        
        # ウィンドウ内のトークンを抽出（キーワードを除く）
        window_tokens = [sentence[i - sentence.start] for i in range(window_start, window_end) if i < span.start or i >= span.end]
        
        # フィルタリングされたトークンのリストを使用して共起語を記録
        record_collocations(window_tokens, collocation_dict, file_name)
        
        # 左側の範囲内のトークンをカウント
        for i, token in enumerate(doc[max(start - range_left, 0):start], start=1):
            token_text = token.text.lower() if lowercase else token.text
            token_index_counts[f"L{range_left - i + 1}"][token_text] += 1

        # キーワードをカウント
        for i, token in enumerate(doc[start:end], start=1):
            token_text = token.text.lower() if lowercase else token.text
            token_index_counts[f"K{i}"][token_text] += 1
        
        # 右側の範囲内のトークンをカウント
        for i, token in enumerate(doc[end:min(end + range_right, len(doc))], start=1):
            token_text = token.text.lower() if lowercase else token.text
            token_index_counts[f"R{i}"][token_text] += 1
    
    return token_index_counts

def record_collocations(tokens, collocation_dict, file_name):
    """
    共起語の情報を辞書に記録する。

    Args:
        tokens (list): 共起語のトークンのリスト
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        file_name (str): 解析対象のファイル名
    """
    for token in tokens:
        key = (token.lemma_, token.pos_)  # トークンのレンマと品詞をキーとして使用
        collocation_dict[key]['lemma'] = token.lemma_  # トークンのレンマを記録
        collocation_dict[key]['pos'] = token.pos_  # トークンの品詞を記録
        collocation_dict[key]['freq'] += 1  # 出現頻度を更新
        collocation_dict[key]['indexes'].append((file_name, token.i))  # トークンの出現位置を記録

def save_collocations_to_csv(collocation_dict, output_file, output_pos=None, max_output=None):
    """
    共起語の情報をCSVファイルに保存する。

    Args:
        collocation_dict (defaultdict): 共起語情報を格納する辞書
        output_file (str): 出力ファイル名
        output_pos (list, optional): 出力する品詞のリスト。デフォルトはNone。
        max_output (int, optional): 出力する結果の最大数。デフォルトはNone。
    """
    # 辞書からデータフレームを作成
    df = pd.DataFrame.from_dict(collocation_dict, orient='index')
    
    # 全品詞に対するソート
    df_sorted_all = df.sort_values(by='freq', ascending=False)
    if max_output is not None:
        df_sorted_all = df_sorted_all.head(max_output)
    
    # 指定された品詞でフィルタリング
    dfs_filtered = []
    if output_pos:
        for pos in output_pos:
            df_filtered = df[df['pos'] == pos].sort_values(by='freq', ascending=False)
            if max_output is not None:
                df_filtered = df_filtered.head(max_output)
            df_filtered = df_filtered[['lemma', 'freq']].reset_index(drop=True)
            df_filtered.columns = [f'lemma_{pos}', f'freq_{pos}']
            dfs_filtered.append(df_filtered)
    
    # 指定された品詞に基づくデータフレームを結合
    df_filtered_combined = pd.concat(dfs_filtered, axis=1)
    # 全品詞の共起語データを追加
    df_final = pd.concat([df_filtered_combined, df_sorted_all[['lemma', 'pos', 'freq']].reset_index(drop=True)], axis=1)
    
    # CSVとして保存
    df_final.to_csv(output_file, index=False)

# 共起語を格納するための辞書の初期化
collocation_dict = defaultdict(lambda: {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []})
all_token_index_counts = defaultdict(lambda: defaultdict(int))  # すべてのドキュメントのトークンのインデックスごとのカウント

# 各文書に対する共起語の検索と記録
for file_name, doc in filtered_docs.items():
    token_index_counts = search_with_window(doc, pattern, collocation_dict, file_name, range_left, range_right)
    # トークンの相対インデックスごとのカウントをマージ
    for index, counts in token_index_counts.items():
        for token, count in counts.items():
            all_token_index_counts[index][token] += count

# 共起語の総数を計算し、結果を表示
total_matches = len(collocation_dict)  # 共起語の総数を計算
if total_matches > 0:
    print(f"分析結果: {total_matches} 件の共起語が見つかりました。")
    save_collocations_to_csv(collocation_dict, output_file, output_pos, max_output)  # 結果をCSVファイルに保存
    print(f"結果が {output_file} に保存されました。")
else:
    print("結果が見つかりませんでした。")

## キーワードや周辺トークンの集計

In [ ]:
# 出力するCSVファイルの名前を定義
output_filename = "indexed_token_counts.csv"

def output_indexed_token_counts(token_index_counts, output_filename):
    """
    トークンの相対インデックスごとの集計結果をデータフレームに変換し、CSVファイルに出力する。

    Args:
        token_index_counts (defaultdict): トークンの相対インデックスごとの出現回数をカウントする辞書。
        output_filename (str): 出力するCSVファイルの名前。

    Returns:
        pandas.DataFrame: トークンの相対インデックスごとの集計結果を含むデータフレーム。
    """

    # 集計データをリストに変換する
    data = []
    for index, tokens in token_index_counts.items():
        for token, count in tokens.items():
            data.append({"Index": index, "Token": token, "Count": count})

    # リストからデータフレームを作成し、インデックスとカウントでソート
    df = pd.DataFrame(data)
    df_sorted = df.sort_values(by=["Index", "Count"], ascending=[True, False])

    # 最終的な結果を格納するための空のデータフレームを作成
    result_df = pd.DataFrame()

    # 相対インデックスの順序を指定
    index_order = [f"L{i}" for i in range(range_left, 0, -1)] + [f"K{i}" for i in range(1, 6)] + [f"R{i}" for i in range(1, range_right + 1)]

    # 指定された順序で相対インデックスごとにデータフレームを作成し、最終結果に追加
    for index in index_order:
        if index in set(df_sorted["Index"]):
            temp_df = df_sorted[df_sorted["Index"] == index].copy()
            temp_df.drop("Index", axis=1, inplace=True)  # インデックス列は不要なので削除
            temp_df.columns = [f"Token_{index}", f"Freq_{index}"]  # 列名を設定
            result_df = pd.concat([result_df, temp_df.reset_index(drop=True)], axis=1)

    # 最終的な結果をCSVファイルに出力
    result_df.to_csv(output_filename, index=False)

    return result_df

# 関数の実行とCSVファイルへの出力
result_df = output_indexed_token_counts(all_token_index_counts, output_filename)

print(f"トークンのインデックス別集計結果を {output_filename} に出力しました。")

# 例文抽出機能
## ユーザーが設定するパラメータ―

In [ ]:
lemma_to_extract = 'earth' # 抽出する単語を指定 
pos_to_extract = 'NOUN'  # 抽出する単語の品詞を指定
output_text_file = 'extract_file.txt'  # 抽出した文を保存するファイル名

## 例文抽出の実行

In [ ]:
def extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma, pos, output_file):
    """
    指定された共起語と品詞に対応する文を抽出し、テキストファイルに保存する。

    Args:
        filtered_docs (dict): フィルタリングされたDocオブジェクトの辞書
        collocation_dict (defaultdict): 共起語情報を格納する辞書(例 : {(lemma, pos): {'lemma': '', 'pos': '', 'freq': 0, 'indexes': []}, ...}
        lemma (str): 抽出する単語のレンマ
        pos (str): 抽出する単語の品詞
        output_file (str): 抽出した文を保存するファイル名
    """
    # 共起語と品詞に対応するインデックスを取得(例 : indexes = [(file_name, token_index), ...]
    indexes = collocation_dict.get((lemma, pos), {}).get('indexes', [])
    if not indexes:
        print(f"警告: 指定された共起語 ({lemma}, {pos}) に対応する例文が見つかりませんでした。")
        return
    
    # 処理された文を追跡するためのセット
    processed_sentences = set()

    # ドキュメントとテキスト情報のペアを用いて実際の文を抽出
    sentences = []
    for file_name, doc in filtered_docs.items():
        for token in doc:
            # トークンのインデックスと文書のテキスト情報を組み合わせたキーを作成
            token_key = (file_name, token.i)
            if token_key in indexes:
                sentence_text = token.sent.text
                # 重複を避けるために、すでに処理された文をチェック
                if sentence_text not in processed_sentences:
                    sentences.append(sentence_text)
                    processed_sentences.add(sentence_text)
    
    # テキストファイルに1行1文の形で出力
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            f.write(sentence + '\n')
    print(f"抽出された文({len(sentences)}件)が {output_file} に保存されました。")

# 文を抽出してテキストファイルに保存
extract_sentences_by_collocation(filtered_docs, collocation_dict, lemma_to_extract, pos_to_extract, output_text_file)